# Question 1

In [ ]:
# Let X be a random vector with cov matrix S. Construct a random vector X = C*U,
# where U is the vector of independent standard normal variables

import datetime as dt

import numpy as np
import pandas as pd
import yfinance as yf

# For our sample covariance matrix S, let's use a collection of AI stocks, and
# look at their returns since the release date of ChatGPT: November 30, 2022
tickers = ["NVDA", "ORCL", "GOOGL", "META", "PLTR", "MSFT", "^GSPC"]
startDate = dt.datetime(2022, 11, 30)
endDate = dt.datetime.now()

# Download price data
prices_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=startDate, end=endDate)
    prices_df[ticker] = data["Close"]

# Calculate stock returns
stock_returns = prices_df.pct_change().dropna()

In [2]:
stock_returns

,NVDA,ORCL,GOOGL,META,PLTR,MSFT,^GSPC
Date,,,,,,,
2022-12-01,0.012528,0.013008,0.000000,0.019814,0.048000,-0.001764,-0.000868
2022-12-02,-0.015116,-0.009036,-0.005446,0.025324,-0.025445,0.001296,-0.001195
2022-12-05,-0.015762,-0.047031,-0.009558,-0.008584,-0.061358,-0.018901,-0.017894
2022-12-06,-0.037507,-0.006547,-0.025131,-0.067875,-0.027816,-0.020304,-0.014399
2022-12-07,0.008319,-0.001648,-0.021035,-0.001665,0.011445,-0.003060,-0.001862
...,...,...,...,...,...,...,...
2025-11-03,0.021680,-0.018126,0.008997,-0.016411,0.033471,-0.001506,0.001721
2025-11-04,-0.039588,-0.037541,-0.021782,-0.016293,-0.079351,-0.005222,-0.011737
2025-11-05,-0.017515,0.008623,0.024393,0.013757,-0.014889,-0.013940,0.003654


In [3]:
# Calculate the covariance matrix from the asset returns
cov_matrix = stock_returns.cov()

# Define a function that constructs the random vector X = C*U
def generate_X(cov_matrix: pd.DataFrame, n_sims: int = 1):

    # Check for the correct input
    if not isinstance(cov_matrix, pd.DataFrame):
        raise TypeError(f"Expected input pandas DataFrame, got {type(cov_matrix).__name__}")

    if not isinstance(n_sims, int):
        raise TypeError(f"Expected input int, got {type(n_sims).__name__}")

    # Generate a vector of standard normal random variables
    U = np.random.normal(size=(cov_matrix.shape[0], n_sims))

    # Use Cholesky decomposition to construct the randm vector X with the
    # given covariance matrix
    C = np.linalg.cholesky(cov_matrix, upper=False)
    X = np.dot(C, U).transpose()

    return X


In [4]:
# Let's test whether the function generate_X works as intended
# We can do this by implementing a simple Monte Carlo simulation
N = 10**7
simulations = generate_X(cov_matrix, N)
sim_data = pd.DataFrame(simulations, columns=cov_matrix.columns)
cov_mc = sim_data.cov()

# Look at the difference between the covariance matrix from the simulation
# vs the original covariance matrix
cov_mc - cov_matrix


,NVDA,ORCL,GOOGL,META,PLTR,MSFT,^GSPC
NVDA,-2.851746e-07,-1.081618e-07,1.929611e-07,-1.309195e-08,-1.464942e-07,1.308296e-07,-7.422474e-09
ORCL,-1.081618e-07,3.338346e-07,1.155577e-07,4.330668e-08,-3.236817e-07,1.088390e-07,6.585519e-10
GOOGL,1.929611e-07,1.155577e-07,9.903254e-08,-4.477160e-08,1.843827e-07,1.610888e-07,3.502409e-08
META,-1.309195e-08,4.330668e-08,-4.477160e-08,-9.696950e-08,-2.074752e-07,4.691167e-08,-7.480612e-08
PLTR,-1.464942e-07,-3.236817e-07,1.843827e-07,-2.074752e-07,-1.435564e-06,-8.097344e-08,-1.480306e-07
MSFT,1.308296e-07,1.088390e-07,1.610888e-07,4.691167e-08,-8.097344e-08,1.091146e-07,2.025282e-08
^GSPC,-7.422474e-09,6.585519e-10,3.502409e-08,-7.480612e-08,-1.480306e-07,2.025282e-08,-3.026634e-08


# Question 2

In [5]:
import QuantLib as ql
import numpy as np
import pandas as pd
from scipy.optimize import root_scalar

## Simple Solution

In [6]:
# Define a simple interpolation algorithm
def naive_rates(maturities: list, term_structure: list) -> float:
    return np.interp([maturities], [t for t, _ in term_structure], [r for _, r in term_structure])[0]


# Helper function to price bonds
# Price assuming the zero rate curve is made of simple rates
def bond_pricer_continuous(payments, paym_mat_rates, paym_maturities, spread):
    present_value = [
        payments[i] * np.exp(-(paym_mat_rates[i] + spread) * paym_maturities[i]) for i in range(len(payments))
    ]
    return np.sum(present_value)


# Price assuming the zero rate curve is made of simple rates
def bond_pricer_simple(payments, paym_mat_rates, paym_maturities, spread):
    present_value = [
        payments[i] / (1 + (paym_mat_rates[i] + spread) * paym_maturities[i]) for i in range(len(payments))
    ]
    return np.sum(present_value)


# Price assuming the zero rate curve is made of simple rates
def bond_pricer_annual(payments, paym_mat_rates, paym_maturities, spread):
    present_value = [
        payments[i] / ((1 + paym_mat_rates[i] + spread) ** paym_maturities[i]) for i in range(len(payments))
    ]
    return np.sum(present_value)


# Create a simple optimization algorithm to find the implied volatility
def naive_solver(func, bounds: tuple, tol: float = 1e-5) -> float:
    """
    Given a function, finds the solution to the function f(x) = 0
    Uses a simple binary search of the input space to find the solution,
    assumes that the function is monotonic in the input.

    Args:
      func (function): The function to solve.
      bounds (tuple): The bounds of the solution space, (lower, upper).
      tol (float, optional): The tolerance for the solution. Defaults to 1e-5.

    Returns:
      float: The solution to the function.
    """

    lower, upper = bounds

    solution = (lower + upper) / 2

    if func(lower) > func(upper):
        # This monotonic function is decreasing
        while abs(func(solution)) > tol:
            if func(solution) < 0:
                upper = solution
            else:
                lower = solution
            solution = (lower + upper) / 2
    else:
        # This monotonic function is increasing
        while abs(func(solution)) > tol:
            if func(solution) > 0:
                upper = solution
            else:
                lower = solution
            solution = (lower + upper) / 2
    return solution


def implied_spread(
    func,
    T: float,
    cpn_r: float,
    cpn_freq: float,
    notional: float,
    price: float,
    term_structure: list,
    bounds: tuple = (0.0001, 0.50),
    tol: float = 1e-5,
) -> float:
    """
    Calculates the implied credit spread of a fixed coupon bond given its
    parameters and a pricing function. Implements a binary search algorithm to
    find the implied spread, which assumes that the price of the bond is monotonic
    in the spread. The function assumes a flat structure of credit spreads. The
    function implements a simple linear interpolation of the term structure of
    zero rates.

    Args:
      func (function): The pricing function of the bond.
      T (float): The time to maturity of the bond.
      cpn_r (float): The coupon rate of the bond.
      cpn_freq (float): The frequency of the coupon payments (p.a.).
      notional (float): The notional value of the bond.
      price (float): The price of the bond.
      term_structure: A list object with tuple pairs (maturity, zero rate).
      bounds (tuple, optional): The bounds of the implied credit spread.

    Return:
      float: The implied credit spread.
    """
    # Define a list of payments dates
    n_periods = int(T / cpn_freq)

    paym_maturities = np.linspace(cpn_freq, T, n_periods, endpoint=True)

    # Define a list of payments
    payments = [
        notional * (1 + cpn_r * cpn_freq) if year == T else notional * cpn_r * cpn_freq
        for year in paym_maturities
    ]

    # Extract interpolated zero rates for the payment maturities
    if not any(t_val == 0 for t_val, _ in term_structure):
        term_structure.insert(0, (0, term_structure[0][1]))

    paym_mat_rates = naive_rates(paym_maturities, term_structure)

    bond_price = lambda s: func(payments, paym_mat_rates, paym_maturities, s)

    return naive_solver(func=lambda x: bond_price(x) - price, bounds=bounds, tol=tol)


In [7]:
# Given parameters
T = 6.5
CPN_R = 0.02
CPN_FREQ = 1 / 4
NOTIONAL = 100
PRICE = 95.3

# Term structure of zero rate: I appended a 0.5% 0 maturity date for interpolation.
TERM_STRUCTURE = [(0, 0.005), (1, 0.005), (3, 0.01), (5, 0.012), (7, 0.014), (10, 0.02)]

# Results
continuous_spread = implied_spread(
    bond_pricer_continuous, T, CPN_R, CPN_FREQ, NOTIONAL, PRICE, TERM_STRUCTURE
)
simple_spread = implied_spread(
    bond_pricer_simple, T, CPN_R, CPN_FREQ, NOTIONAL, PRICE, TERM_STRUCTURE
)
annual_comp_spread = implied_spread(
    bond_pricer_annual, T, CPN_R, CPN_FREQ, NOTIONAL, PRICE, TERM_STRUCTURE
)

# Print results
print(f"The implied credit spread with continuous zero rates is {100*continuous_spread:.2f}%")
print(f"The implied credit spread with simple zero rates is {100*simple_spread:.2f}%")
print(f"The implied credit spread with annually compounded zero rates is {100*annual_comp_spread:.2f}%")


The implied credit spread with continuous zero rates is 1.45%
The implied credit spread with simple zero rates is 1.71%
The implied credit spread with annually compounded zero rates is 1.49%


## Implementing QuantLib

In [8]:
# Consider a fixed coupon bond with the following terms and conditions
maturity = 6.5
c_rate = 0.02
freq = 1 / 4
notional = 100
price = 95.3

# Given term structure of zero rates, added a 0 time for interpolation
mat = [0, 1, 3, 5, 7, 10]
rates = [0.005, 0.005, 0.01, 0.012, 0.014, 0.02]

# Reference (today) date.
today = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = today

# Define maturities as period
periods = [ql.Period(year, ql.Years) for year in mat]

In [9]:
# Build calendar dates from periods
calendar = ql.TARGET()
maturity_dates = [calendar.advance(today, p) for p in periods]

# Build a linear zero rate curve
day_count = ql.Actual365Fixed()
curve = ql.ZeroCurve(maturity_dates, rates, day_count, calendar, ql.Linear())

In [10]:
# Extract relevant interpolated continuous rates for maturities of cash flows
n_periods = int(maturity / freq)
paym_maturities = np.linspace(freq, maturity, n_periods, endpoint=True)

# Create a helper function to extract interpolated rates
def get_rate(period):
    target_date = calendar.advance(today, ql.Period(int(period * 12), ql.Months))
    zero_rate = curve.zeroRate(target_date, day_count, ql.Continuous).rate()
    return zero_rate


# Create a list of interpolated rates for payment maturities
paym_mat_rates = [get_rate(period) for period in paym_maturities]

# Create a list of payments
payments = [
    notional * (1 + c_rate * freq) if year == maturity else notional * c_rate * freq
    for year in paym_maturities
]

# Create a helper function to price a bond
def price_bond(payments, paym_mat_rates, paym_maturities, spread):
    present_value = [
        payments[i] * np.exp(-(paym_mat_rates[i] + spread) * paym_maturities[i]) for i in range(len(payments))
    ]
    return np.sum(present_value)


In [11]:
# Find the implied credit spread by optimization
def difference(spread):
    return price_bond(payments, paym_mat_rates, paym_maturities, spread) - price


implied_spread = root_scalar(difference, bracket=[0.01, 0.02])
implied_spread.root
print(f"The implied credit spread with continuous zero rates using QuantLib is {100*implied_spread.root:.2f}%")

The implied credit spread with continuous zero rates using QuantLib is 1.45%


# Question 3

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
def american_option(P_0: float,
                    K: float,
                    sigma: float,
                    maturity: float,
                    term_structure: list,
                    option_type: str = "put",
                    n: int = 200,
) -> float:
  '''
  Calculates the value of an american option using a recursive binomial tree
  which is constructed using the given parameters. The zero rate term structure
  is fitted to the tree using a linear interpolation algorithm, and assuming
  deterministic forward rates. The zero rates given are assumed to be continuous.

  It is possible to calculate the value of both a call and a put option.

  Args:
    P_0 (float): The initial price of the underlying asset.
    K (float): The strike price of the option.
    sigma (float): The annualized volatility of the underlying asset.
    maturity (float): The time to maturity of the option.
    term_structure: A list object with tuple pairs (maturity, zero rate).
    option_type (str, optional): The type of option to calculate. Defaults to "put".
    n (int, optional): The number of time steps in the binomial tree. Defaults to 200.

  Returns:
    float: The value of the american option.
  '''
  if not any(t_val == 0 for t_val, _ in term_structure):
    term_structure.insert(0, (0, term_structure[0][1]))

  if option_type not in ["put", "call"]:
    raise ValueError(f"option_type expected input: ['put', 'call'], given input: {option_type}")

  option_type = -1 if option_type == "put" else 1

  # Define helper functions for and calculate the interpolated forward rates
  # Define a simple interpolation algorithm
  def naive_rates(time: float, term_structure: list) -> float:
    return np.interp([time], [t for t, _ in term_structure], [r for _, r in term_structure])[0]

  # Simple forward rates (continuous zero rates)
  def naive_forward_rates(start_t: float, end_t: float, term_structure: list) -> float:
    return (naive_rates(end_t, term_structure)*end_t - naive_rates(start_t, term_structure)*start_t)/(end_t - start_t)

  # Initialize parameters for the binomial tree
  dt = maturity / n
  u = np.exp(sigma*np.sqrt(dt))
  d = 1/u

  # Create an array for the forward rates, probabilities, and discount factors
  r = np.array([naive_forward_rates(k*dt, (k+1)*dt, term_structure) for k in range(n)])
  p = (np.exp(r*dt) - d) / (u - d)
  discFct = np.exp(-r*dt)

  # Initiate a binomial tree for the stock price and the american option
  S = np.zeros(shape = (n+1, n+1))
  AmP = np.zeros(shape=S.shape)

  # Populate the binomial tree for the stock price
  for col in range(n+1):
    for row in range(col+1):
      S[row, col] = P_0 * u**(col-row) * d**row

  # Terminal payoff of the american option
  AmP[:, -1] = np.maximum(option_type*(S[:,-1] - K), 0)

  # Recursively calculate the value of the american option
  for i in range(n):
    for row in range(n-i):
      prob = p[n-i-1]
      AmP[row, n-i-1] = np.maximum( discFct[n-i-1]*( prob*AmP[row, n-i] + (1-prob)*AmP[row+1, n-i]), np.maximum(option_type*(S[row, n-i-1] - K), 0))

  return AmP[0,0]


In [14]:
# Create a simple optimization algorithm to find the implied volatility
def naive_solver(func, bounds: tuple, tol: float = 1e-5) -> float:
  '''
  Given a function, finds the solution to the function f(x) = 0
  Uses a simple binary search of the input space to find the solution,
  assumes that the function is monotonic in the input.

  Args:
    func (function): The function to solve.
    bounds (tuple): The bounds of the solution space, (lower, upper).
    tol (float, optional): The tolerance for the solution. Defaults to 1e-5.

  Returns:
    float: The solution to the function.
  '''

  lower, upper = bounds

  solution = (lower + upper)/2

  if func(lower) > func(upper):
    # This monotonic function is decreasing
    while abs(func(solution)) > tol:
      if func(solution) < 0:
        upper = solution
      else:
        lower = solution
      solution = (lower + upper)/2
  else:
    # This monotonic function is increasing
    while abs(func(solution)) > tol:
      if func(solution) > 0:
        upper = solution
      else:
        lower = solution
      solution = (lower + upper)/2
  return solution


In [15]:
# Calculate the implied volatility
def implied_volatility(
    func,
    V_0: float,
    P_0: float,
    K: float,
    maturity: float,
    term_structure: list,
    option_type: str,
    n: int = 400,
    bounds: tuple = (0.01, 2),
    tol: float = 1e-5,
) -> float:
    """
    Calculates the implied volatility of an american option.

    Args:
        func (function): The option pricing function.
        V_0 (float): The current value of the option.
        P_0 (float): The initial price of the underlying asset.
        K (float): The strike price of the option.
        maturity (float): The maturity of the option.
        term_structure: A list object with tuple pairs (maturity, zero rate).
        option_type (str, optional): The type of option to calculate. Defaults to "put".
        n (int, optional): The number of time steps in the binomial tree. Defaults to 500.
        bounds (tuple, optional): The bounds of the implied volatility. Defaults to (0.01,2).
        tol (float, optional): The tolerance for the implied volatility. Defaults to 1e-5.

    Returns:
        float: The implied volatility of the option.
    """
    price_option = lambda s: func(
        sigma=s, P_0=P_0, K=K, maturity=maturity, term_structure=term_structure, option_type=option_type, n=n
    )

    return naive_solver(func=lambda x: price_option(x) - V_0, bounds=bounds, tol=tol)


In [18]:
# Given inputs
# American Option
MATURITY = 1.6
K = 100
P_0 = 120
V_0 = 24.4359
sigma = 0.65

# Term structure
TERM_STRUCTURE = [
    # (0, 0.005),
    (1, 0.005),
    (3, 0.01),
    (5, 0.012),
    (7, 0.014),
    (10, 0.02),
]

options = ["call", "put"]
imp_vol = {}

for option in options:
    imp_vol[option] = implied_volatility(american_option, V_0, P_0, K, MATURITY, TERM_STRUCTURE, option)

print(f"The implied volatility for a Call Option is: {100*imp_vol["call"]:.2f}%")
print(f"The implied volatility for a Put Option is: {100*imp_vol["put"]:.2f}%")

The implied volatility for a Call Option is: 19.68%
The implied volatility for a Put Option is: 63.01%
